In [37]:
import torch

# Checking the installed PyTorch version and CUDA version
pytorch_version = torch.__version__
cuda_version = torch.version.cuda

pytorch_version, cuda_version

('2.1.1+cu121', '12.1')

In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import cv2
import numpy as np
from feat import Detector
import glob
import os

In [39]:
# Initialize the detector

detector = Detector(
    face_model="retinaface",
    landmark_model="mobilefacenet",
    au_model='xgb',
    emotion_model="resmasknet",
    facepose_model="img2pose",
    device="cuda",
)

In [40]:
# Initialize the DataFrame to store AU data
aus_data = pd.DataFrame()

# Define the path to the dataset directory
dataset_path = "../Desktop/kaggle_testset/"

# Loop through each emotion subfolder in the dataset directory
for emotion_folder in os.listdir(dataset_path):
    emotion_path = os.path.join(dataset_path, emotion_folder)

    # Check if it's a directory
    if os.path.isdir(emotion_path):
        # Process each image in the subfolder
        for filename in glob.glob(os.path.join(emotion_path, "*.jpg")):
            img = cv2.imread(filename)
            detection = detector.detect_image(filename)

            # Extract AU data
            aus = detection.aus
            au_data = pd.DataFrame(aus)

            # Add emotion label from folder name
            au_data['emotion'] = emotion_folder

            # Rearrange columns to have 'emotion' first
            cols = ['emotion'] + [col for col in au_data.columns if col != 'emotion']
            au_data = au_data[cols]

            # Append to the main DataFrame
            aus_data = pd.concat([aus_data, au_data], ignore_index=True)

# Save the AU data to a CSV file
aus_data.to_csv("aus_kaggle_set.csv", index=False)

100%|██████████| 1/1 [00:00<00:00,  1.88it/s]
C:\Users\lasse\AppData\Local\Temp\ipykernel_15792\1527144478.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aus_data = pd.concat([aus_data, au_data], ignore_index=True)
100%|██████████| 1/1 [00:00<00:00,  1.94it/s]
C:\Users\lasse\AppData\Local\Temp\ipykernel_15792\1527144478.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aus_data = pd.concat([aus_data, au_data], ignore_index=True)
100%|██████████| 1/1 [00:00<00:00,  2.89it/s]
C:\Users\las

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

# Load the dataset
data = pd.read_csv('aus.csv')

# Ensure that only numeric columns are included in the mean calculation
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Encoding the 'emotion' column
label_encoder = LabelEncoder()
data['emotion'] = label_encoder.fit_transform(data['emotion'])

# Split the dataset into train/validation/test sets (70/20/10 split)
labels = data['emotion']
features = data.drop('emotion', axis=1)

# Splitting the data
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.1, random_state=42, stratify=labels)


# Feature Scaling for AUs
scaler = StandardScaler()
AU_columns = [col for col in data.columns if col.startswith('AU')]
scaler.fit(train_features)
train_features[AU_columns] = scaler.transform(train_features)

scaler.fit(test_features)
test_features[AU_columns] = scaler.transform(test_features)

# Grid search for SVM
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(train_features, train_labels)

# Your previous code remains the same up to the grid search part.

# Best parameters from Grid Search
best_params = grid_search.best_params_

# Create a new SVM model with the best parameters
final_model = SVC(kernel=best_params['kernel'], C=best_params['C'], gamma=best_params['gamma'])

# Train the model on the combined dataset
final_model.fit(train_features, train_labels)

# Evaluate on the test set
test_accuracy = final_model.score(test_features, test_labels)

print(f"Final Model Test Set Accuracy: {test_accuracy}")



In [ ]:
import pickle

# Save the trained model to a file
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(final_model, file)

# Optionally, you can also save the scaler and label encoder if you need them for preprocessing in the future
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)


In [ ]:
import pickle
# Load the model from the file
with open('svm_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Load the scaler and label encoder if needed
with open('scaler.pkl', 'rb') as file:
    loaded_scaler = pickle.load(file)

with open('label_encoder.pkl', 'rb') as file:
    loaded_label_encoder = pickle.load(file)


In [ ]:
detector = Detector()
path = "aaaaaaaa_6.png"
img = cv2.imread(path)

In [ ]:
detection = detector.detect_image(path)
detection.aus

In [ ]:
%%timeit
faces = detector.detect_faces(img)
landmarks = detector.detect_landmarks(img, detected_faces=faces)
new_au_data = detector.detect_aus(img, landmarks=landmarks)


In [ ]:
%%timeit
faces = detector.detect_faces(img)

In [ ]:
%%timeit
landmarks = detector.detect_landmarks(img, detected_faces=faces)

In [ ]:
%%timeit
new_au_data = detector.detect_aus(img, landmarks=landmarks)


In [ ]:
# Example: Assuming 'new_au_data' is your extracted AU features from Py-FEAT
new_au_data_scaled = loaded_scaler.transform(new_au_data)  # Use the same scaler as during training
predictions = loaded_model.predict(new_au_data_scaled)
predicted_emotions = loaded_label_encoder.inverse_transform(predictions)

predicted_emotions[0]